In [ ]:
import numpy as np
import cv2
import os

from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


VIT

In [ ]:
def load_npy_files(image_npy_path, mask_npy_path):
    images = np.load(image_npy_path)
    masks = np.load(mask_npy_path)
    return images, masks

def data_augmentation(images, masks, batch_size=8):
    data_gen_args = dict(rotation_range=20,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.1,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode='nearest')
    
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    image_datagen.fit(images, augment=True, seed=1)
    mask_datagen.fit(masks, augment=True, seed=1)

    image_generator = image_datagen.flow(images, batch_size=batch_size, seed=1)
    mask_generator = mask_datagen.flow(masks, batch_size=batch_size, seed=1)

    return zip(image_generator, mask_generator)

def visualize_augmented_data(images, masks, num_images_to_show=3):
    for i in range(num_images_to_show):
        image, mask = images[i], masks[i]
        image_augmented, mask_augmented = next(data_augmentation(np.array([image]), np.array([mask])))

        cv2_imshow(image)
        cv2_imshow(mask)
        cv2_imshow(image_augmented[0])
        cv2_imshow(mask_augmented[0])

        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    image_npy_path = '/content/drive/MyDrive/EECS545 Final Project/train_input0.npy' # Since i don't have Mingyu's label files so I used face.npy in hw5 to check my code
    mask_npy_path = '/content/drive/MyDrive/EECS545 Final Project/train_label0.npy'

    images, masks = load_npy_files(image_npy_path, mask_npy_path)

    images = np.expand_dims(images, axis=-1)  
    masks = np.expand_dims(masks, axis=-1)

    visualize_augmented_data(images, masks)

ValueError: ignored

segmentation

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    # ... [add more layers following the U-Net architecture] ...

    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u8) # this part is incorrect, it seems that u8 in colab doesn't work
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9) # this part is incorrect, same error as above
    c9 = BatchNormalization()(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9) # this part is incorrect, same error as above

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

def train_unet(images, masks, epochs=50, batch_size=8):
    model = unet_model(input_size=(images.shape[1], images.shape[2], images.shape[3]))

    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    model_checkpoint = ModelCheckpoint('unet_weights.hdf5', monitor='val_loss', save_best_only=True)
    early_stopping = EarlyStopping(patience=10, verbose=1)

    model.fit(images, masks, batch_size=batch_size, epochs=epochs, verbose=1,
              validation_split=0.2, shuffle=True, callbacks=[model_checkpoint, early_stopping])

    return model

if __name__ == "__main__":
    image_npy_path = 'face.npy' 
    mask_npy_path = 'face.npy'

    images, masks = load_npy_files(image_npy_path, mask_npy_path)

    # Normalize images and masks to [0, 1] range
    images = images.astype(np.float32) / 255.0
    masks = masks.astype(np.float32) / 255.0

    # Train U-Net model
    model = train_unet(images, masks)